In [4]:
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA

# Load data from JSON file
with open("data.json") as f:
    data = json.load(f)

# Extract features and target
t = np.array(data["t"])
o = np.array(data["o"])
h = np.array(data["h"])
l = np.array(data["l"])
c = np.array(data["c"])
v = np.array(data["v"])

# Use only "c" as target
y = c

# Normalize data
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Split data into train and test sets
y_train, y_test = train_test_split(y_scaled, test_size=0.5, random_state=42)

# Convert to 1D array
y_train = y_train.flatten()
y_test = y_test.flatten()

# Build and train the ARIMA model
order = (5, 1, 0)  # Example order, you may need to tune this
model = ARIMA(y_train, order=order)
model_fit = model.fit()

# Evaluate the model on test data
predictions = model_fit.forecast(steps=len(y_test))
predictions = predictions.reshape(-1, 1)
predictions_inverse = scaler_y.inverse_transform(predictions)
test_inverse = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Mean Squared Error
mse = np.mean((predictions_inverse - test_inverse) ** 2)
print("Test MSE:", mse)

# Predict future values
future_steps = 7
future_forecast = model_fit.forecast(steps=future_steps)
future_forecast = future_forecast.reshape(-1, 1)
future_forecast_inverse = scaler_y.inverse_transform(future_forecast)
print("Predicted Close Prices for Next 7 Days:", future_forecast_inverse)

Test MSE: 180589.31870997223
Predicted Close Prices for Next 7 Days: [[ 897.62502778]
 [ 984.15700959]
 [1003.18300581]
 [ 889.11231005]
 [ 817.39540762]
 [ 819.51960623]
 [ 909.74547833]]
